In [34]:
import json
import pandas as pd
import cv2
import numpy as np
from enum import Enum, auto
import sys
import torch

sys.path.append("../Skin_segmentation")
sys.path.append('../../Models')

from Skindetector import *
from rPPGNet import rPPGNet

with open('../../Data/json_structure') as json_file:
    data = json.load(json_file)


root_dir = "/work3/s174159/data/"
video_file = root_dir + "00/01/c920-1.avi"
bb_file = root_dir + "bbox/00/01/c920-1.face"
ecg_path = root_dir + "00/01/c920-1.vitacom"
index_path = root_dir + "00/01/c920-1.csv"

bb_data =  pd.read_csv(bb_file, sep=" ", header=None, names=["frame", "x", "y", "w", "h"]).drop("frame", axis=1)

In [2]:
model = rPPGNet()
model.load_state_dict(torch.load("../../logs/Eulerian/models/run_2023-10-25_15-30-02/model.pth", map_location=torch.device('cpu')))
model.eval()

rPPGNet(
  (ConvSpa1): Sequential(
    (0): Conv3d(3, 16, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (ConvSpa3): Sequential(
    (0): SpatioTemporalConv(
      (spatial_conv): Conv3d(16, 57, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
      (bn): BatchNorm3d(57, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (temporal_conv): Conv3d(57, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
    )
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (ConvSpa4): Sequential(
    (0): SpatioTemporalConv(
      (spatial_conv): Conv3d(32, 72, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
      (bn): BatchNorm3d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      

In [4]:
import cv2
from IPython.display import display, Image

# Define the VideoCapture object and VideoWriter
output_file = 'output_video.mp4'
video = cv2.VideoCapture(video_file)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for the output video
out = cv2.VideoWriter(output_file, fourcc, 30, (128, 128) ) # 30 is the frames per second, (128, 128) is the frame size

display_handle = display(None, display_id=True)
i = 0

bounded_frames = []

try:
    while True:
        ret, frame = video.read()
        if not ret:  # Check if there are no more frames
            print("Finished video...")
            break  # Exit the loop if there are no more frames
        
        if i == len(data):
            i = 0
        x, y, w, h = bb_data.iloc[i].values.astype(int)
        i += 1
        frame_bb = frame[y:y + h, x:x + w]
        frame_bb = cv2.resize(frame_bb, (128, 128), interpolation=cv2.INTER_AREA)
        bounded_frames.append(np.array(frame_bb, dtype='float32'))
        #detector = SkinDetector(frame_bb)
        #detector.find_skin()
        #image, mask, skin = detector.get_resulting_images()
        
        # Save the processed frame to the output video
        #out.write(frame)
        
        #_, frame = cv2.imencode('.jpeg', frame)
        #display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    out.release()  # Release the VideoWriter
    display_handle.update(None)


None

Finished video...


In [27]:
frame_tensor = torch.tensor(np.array(bounded_frames[1:65]))
frame_tensor = torch.swapaxes(frame_tensor, 0, 3)
frame_tensor = torch.swapaxes(frame_tensor, 1, 3)

In [28]:
skin_map, rPPG_aux, rPPG, rPPG_SA1, rPPG_SA2, rPPG_SA3, rPPG_SA4, x_visual6464, x_visual3232  = model(frame_tensor.unsqueeze(0))

In [31]:
rPPG[0]

tensor([-0.1357, -0.1362, -0.1356, -0.1359, -0.1360, -0.1361, -0.1361, -0.1361,
        -0.1362, -0.1362, -0.1362, -0.1363, -0.1363, -0.1363, -0.1364, -0.1366,
        -0.1369, -0.1374, -0.1378, -0.1380, -0.1381, -0.1382, -0.1381, -0.1381,
        -0.1381, -0.1381, -0.1381, -0.1380, -0.1380, -0.1379, -0.1378, -0.1377,
        -0.1378, -0.1380, -0.1384, -0.1388, -0.1390, -0.1390, -0.1390, -0.1388,
        -0.1387, -0.1385, -0.1384, -0.1383, -0.1381, -0.1380, -0.1377, -0.1375,
        -0.1373, -0.1372, -0.1373, -0.1375, -0.1377, -0.1377, -0.1377, -0.1375,
        -0.1372, -0.1369, -0.1366, -0.1363, -0.1360, -0.1356, -0.1349, -0.1341],
       grad_fn=<SelectBackward0>)

In [30]:
frame_tensor.unsqueeze(0).shape

torch.Size([1, 3, 64, 128, 128])